In [1]:
import time
import blot
import numpy as np
import pandas as pd

from datetime import datetime, timedelta
from dateutil import parser

In [2]:
raw_data = pd.read_csv("../data/storage.csv")
raw_data["time"] = pd.to_datetime(raw_data["time"], unit="s")
# data["time"] =  data["time"].apply(lambda ts: datetime.fromtimestamp(ts))
raw_data = raw_data.set_index(["time"])
raw_data["value"] = raw_data["value"] / 1000
# data = data.loc[(data.index > datetime(2020, 8, 30)) & (data.index < datetime(2020, 9, 30))]
data = raw_data.loc["2020-9-11": "2020-9-18"]
data.blot(theme="white", color="blue")

In [3]:
data["value"].values.flatten()

array([0.81 , 0.866, 0.962, ..., 0.678, 0.675, 0.695])

In [4]:
# 直接使用 statsmodels 库
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing

decompose_result = seasonal_decompose(data, model="multiplicative", period=288)
fig = decompose_result.seasonal.blot(theme="white", colors="blue") + decompose_result.trend.blot(theme="white", color="red") + decompose_result.resid.blot(theme="white", color="green")
fig.show()


In [5]:
# 参数
k = 288
alpha = 0.5

print(pd.infer_freq(data.index))
# data.index.freq = 'T'
# data.index.freq = "5min"

None


In [6]:
## 一次指数平滑
data["1exp"] = SimpleExpSmoothing(data["value"]).fit(smoothing_level=alpha).fittedvalues
data.blot(theme="white")

/Users/shenmengjia/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:583: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/Users/shenmengjia/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/holtwinters/model.py:429: FutureWarning:

After 0.13 initialization must be handled at model creation



In [7]:
data["2exp_add"] = ExponentialSmoothing(data["value"], trend="add", seasonal=None).fit().fittedvalues
data["2exp_mul"] = ExponentialSmoothing(data["value"], trend="mul", seasonal=None).fit().fittedvalues
data[["value", "2exp_add", "2exp_mul"]].blot(theme="white")

/Users/shenmengjia/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:583: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/Users/shenmengjia/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:583: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



In [8]:
data["3exp_add"] = ExponentialSmoothing(data["value"], trend="add", seasonal="add", seasonal_periods=288).fit().fittedvalues
data["3exp_mul"] = ExponentialSmoothing(data["value"], trend="mul", seasonal="mul", seasonal_periods=288).fit().fittedvalues
data[["value", "3exp_add", "3exp_mul"]].blot(theme="white")

/Users/shenmengjia/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:583: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/Users/shenmengjia/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:583: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/Users/shenmengjia/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/holtwinters/model.py:80: RuntimeWarning:

overflow encountered in matmul

/Users/shenmengjia/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



In [9]:
## 预测
model = ExponentialSmoothing(data["value"], trend="add", seasonal="add", seasonal_periods=288).fit()
pred = model.forecast(288)

/Users/shenmengjia/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:583: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/Users/shenmengjia/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:379: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.



In [10]:
raw_data.blot()

In [13]:
true = raw_data.loc["2020-9-23": "2020-9-23"]
true = true[:len(true)-2]
true["pred"] = pred.values[2:]
true.blot(theme="white")